In [1]:
%cd /Users/hatakeyamaatsushi/PycharmProjects/movie_contents_app

/Users/hatakeyamaatsushi/PycharmProjects/movie_contents_app


In [2]:
from app import db
from app.playlists.models import Playlist
from app.users.models import User
from app.videos.models import Video
from cassandra.cqlengine.management import sync_table

In [3]:
from app.indexing.client import get_index, update_index
from app.indexing.schemas import *

In [4]:
db.get_session()
sync_table(User)
sync_table(Video)
sync_table(Playlist)

In [5]:
import uuid
import json
from pydantic import BaseModel, Field , validator, root_validator
from typing import Optional

In [6]:
video_obj = dict(Video.objects.first())
video_obj

{'host_id': '3kYkEVIZNZY',
 'db_id': UUID('0f4e5560-9190-11ec-b764-acde48001122'),
 'host_service': 'youtube',
 'title': 'next is',
 'url': 'https://www.youtube.com/watch?v=3kYkEVIZNZY',
 'user_id': UUID('f2ae4380-7e88-11ec-8138-acde48001122')}

In [7]:
class VideoIndexSchema2(BaseModel):
    objectID: str = Field(alias='host_id')
    objectType: str = "Video"
    title:Optional[str] 
    path: str = Field(alias='host_id')
        
    @validator("path")
    def set_path(cls, v, values, **kwargs):
        host_id = v
        return f"/videos/{host_id}"

In [8]:
VideoIndexSchema(**video_obj).dict()

{'objectID': '3kYkEVIZNZY',
 'objectType': 'Video',
 'title': 'next is',
 'path': '/videos/3kYkEVIZNZY'}

In [9]:
playlist_obj = dict(Playlist.objects.first())
playlist_obj

{'db_id': UUID('e0bb0fa8-97d5-11ec-a98f-acde48001122'),
 'user_id': UUID('f2ae4380-7e88-11ec-8138-acde48001122'),
 'updated': datetime.datetime(2022, 2, 27, 14, 3, 21, 850000),
 'host_ids': ['SLB_c_ayRMo'],
 'title': 'python'}

In [10]:
class PlaylistIndexSchema2(BaseModel):
    objectID: uuid.UUID = Field(alias='db_id')
    objectType: str = "Playlist"
    title:Optional[str] 
    path: str = Field(default='/')
        
#     @validator("path")
#     def set_path(cls, v, values, **kwargs):
#         db_id = v
#         return f"/playlists/{db_id}"
    
    @root_validator
    def set_default(cls, values):
        objectID = values.get('objectID')
        values['objectID'] = str(objectID)
        values['path'] = f"/playlist/{objectID}"
        return values

In [11]:
json.loads(PlaylistIndexSchema(**playlist_obj).json())

{'objectID': 'e0bb0fa8-97d5-11ec-a98f-acde48001122',
 'objectType': 'Playlist',
 'title': 'python',
 'path': '/playlist/e0bb0fa8-97d5-11ec-a98f-acde48001122'}

In [12]:
PlaylistIndexSchema(**playlist_obj).dict()

{'objectID': 'e0bb0fa8-97d5-11ec-a98f-acde48001122',
 'objectType': 'Playlist',
 'title': 'python',
 'path': '/playlist/e0bb0fa8-97d5-11ec-a98f-acde48001122'}

In [13]:
video_q = [dict(x) for x in Video.objects.all()]
videos_dataset = [VideoIndexSchema(**x).dict() for x in video_q]
videos_dataset

[{'objectID': '3kYkEVIZNZY',
  'objectType': 'Video',
  'title': 'next is',
  'path': '/videos/3kYkEVIZNZY'},
 {'objectID': 'azcrPFhaY9k',
  'objectType': 'Video',
  'title': 'thinker revised',
  'path': '/videos/azcrPFhaY9k'},
 {'objectID': 'fAAZixBzIAI',
  'objectType': 'Video',
  'title': 'algorithm renamed change',
  'path': '/videos/fAAZixBzIAI'},
 {'objectID': 'e26zZ83Oh6Y',
  'objectType': 'Video',
  'title': 'guitar play',
  'path': '/videos/e26zZ83Oh6Y'},
 {'objectID': 'nTeuhbP7wdE',
  'objectType': 'Video',
  'title': 'react study tool',
  'path': '/videos/nTeuhbP7wdE'},
 {'objectID': 'TxM8p5F6DXE',
  'objectType': 'Video',
  'title': 'how to study programs',
  'path': '/videos/TxM8p5F6DXE'},
 {'objectID': 'PrcZXRD8pBc',
  'objectType': 'Video',
  'title': 'space brother',
  'path': '/videos/PrcZXRD8pBc'},
 {'objectID': 'blEApq7Qa-Q',
  'objectType': 'Video',
  'title': 'while practice',
  'path': '/videos/blEApq7Qa-Q'},
 {'objectID': 'UKdQjQX1Pko',
  'objectType': 'Video',
 

In [14]:
playlist_q = [dict(x) for x in Playlist.objects.all()]
playlists_dataset = [PlaylistIndexSchema(**x).dict() for x in playlist_q]
playlists_dataset

[{'objectID': 'e0bb0fa8-97d5-11ec-a98f-acde48001122',
  'objectType': 'Playlist',
  'title': 'python',
  'path': '/playlist/e0bb0fa8-97d5-11ec-a98f-acde48001122'},
 {'objectID': 'cf899bde-8d24-11ec-b0b7-acde48001122',
  'objectType': 'Playlist',
  'title': 'Video model',
  'path': '/playlist/cf899bde-8d24-11ec-b0b7-acde48001122'},
 {'objectID': '9ee751be-8d25-11ec-b2ce-acde48001122',
  'objectType': 'Playlist',
  'title': 'beautiful landscape',
  'path': '/playlist/9ee751be-8d25-11ec-b2ce-acde48001122'},
 {'objectID': '48f36c4c-8ca4-11ec-93ad-acde48001122',
  'objectType': 'Playlist',
  'title': 'hello world',
  'path': '/playlist/48f36c4c-8ca4-11ec-93ad-acde48001122'},
 {'objectID': 'd47561be-8d24-11ec-b0b7-acde48001122',
  'objectType': 'Playlist',
  'title': 'Videos model',
  'path': '/playlist/d47561be-8d24-11ec-b0b7-acde48001122'}]

In [15]:
dataset = playlists_dataset + videos_dataset

In [16]:
dataset

[{'objectID': 'e0bb0fa8-97d5-11ec-a98f-acde48001122',
  'objectType': 'Playlist',
  'title': 'python',
  'path': '/playlist/e0bb0fa8-97d5-11ec-a98f-acde48001122'},
 {'objectID': 'cf899bde-8d24-11ec-b0b7-acde48001122',
  'objectType': 'Playlist',
  'title': 'Video model',
  'path': '/playlist/cf899bde-8d24-11ec-b0b7-acde48001122'},
 {'objectID': '9ee751be-8d25-11ec-b2ce-acde48001122',
  'objectType': 'Playlist',
  'title': 'beautiful landscape',
  'path': '/playlist/9ee751be-8d25-11ec-b2ce-acde48001122'},
 {'objectID': '48f36c4c-8ca4-11ec-93ad-acde48001122',
  'objectType': 'Playlist',
  'title': 'hello world',
  'path': '/playlist/48f36c4c-8ca4-11ec-93ad-acde48001122'},
 {'objectID': 'd47561be-8d24-11ec-b0b7-acde48001122',
  'objectType': 'Playlist',
  'title': 'Videos model',
  'path': '/playlist/d47561be-8d24-11ec-b0b7-acde48001122'},
 {'objectID': '3kYkEVIZNZY',
  'objectType': 'Video',
  'title': 'next is',
  'path': '/videos/3kYkEVIZNZY'},
 {'objectID': 'azcrPFhaY9k',
  'objectTyp

In [17]:
index = get_index()

In [18]:
# dir(index)

In [24]:
# idx_response =  index.save_objects(dataset).wait()
# len(idx_response[0]['objectIDs'])

updated_count = update_index()
updated_count

32

In [20]:
# index.search("Awesome Open CV")

{'hits': [],
 'nbHits': 0,
 'page': 0,
 'nbPages': 0,
 'hitsPerPage': 20,
 'exhaustiveNbHits': True,
 'exhaustiveTypo': True,
 'query': 'Awesome Open CV',
 'params': 'query=Awesome+Open+CV',
 'renderingContent': {},
 'processingTimeMS': 1}